In [83]:
# pip install pandas seaborn pandasql

In [84]:
import pandas as pd
from pandasql import sqldf

# reading jsonl file to dataframe
df_jsonl = pd.read_json('./data/Jobs_NYC_Postings.jsonl', orient='records', lines=True)
pd.get_option("display.max_columns")
pd.get_option("display.max_colwidth")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

df_jsonl[0:1]

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Career Level,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,606346,DEPARTMENT FOR THE AGING,External,1,Business Operations Analyst,ASSOCIATE STAFF ANALYST,Competitive-1,12627,00,"Administration & Human Resources Technology, Data & Innovation Policy, Research & Analysis",F,Experienced (non-manager),70611.0,81203.0,Annual,"2 Lafayette St., N.Y.",Human Resources,The Office of Human Resources (OHR) seeks a highly motivated individual to serve as a Business O...,"1. A masterâs degree from an accredited college or university, accredited by regional, nationa...",â¢ Project management experience and expertise is a plus. â¢ Self-starter; works independently...,0,Please be sure to submit a resume & cover letter when applying. All current City Employees may a...,0,0,0,"New York City residency is generally required within 90 days of appointment. However, City Emplo...",09/28/2023,27-DEC-2023,09/29/2023,11/20/2023


In [85]:
def exec_query():
    query1 = """
    select *
    from df_jsonl
    where [Salary Range From] > 100000
    limit 2;
    """
    result1 = sqldf(query1, locals())
    return result1

In [86]:
# exec_query()
# error will occur if locals() is used in the function

In [87]:
def exec_query():
    query1 = """
    select *
    from df_jsonl
    where [Salary Range From] > 100000
    limit 2;
    """
    result1 = sqldf(query1, globals())
    return result1

In [88]:
df_job_post = exec_query()
df_job_post[0:1]

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Career Level,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,607141,DEPARTMENT OF CITY PLANNING,External,1,Senior Software Engineer,COMPUTER SPECIALIST (SOFTWARE),Competitive-1,13632,02,"Technology, Data & Innovation",F,Experienced (non-manager),107281.0,120190.0,Annual,"120 Broadway, New York, NY",ITD-DIGITAL SERVICES,THE AGENCY The Department of City Planning (DCP) plans for the strategic growth and development ...,"(1) A baccalaureate degree from an accredited college, including or supplemented by twenty-four ...",â¢\tBuilding RESTful APIs using the Swagger/OpenAPI specification â¢\tCommon Infrastructure-as...,0,PLEASE NOTE: Only applicants under consideration will be contacted. Appointments are subject to...,0,0,0,New York City Residency is not required for this position,09/25/2023,0,10/06/2023,11/20/2023


In [89]:
df_skill = pd.read_csv('./data/technology_skill_30_by_job.csv')
df_skill[0:3]

,similarity_score,technology_skill,job_id_title
0,0.833890,"(5625, 'Oracle HRIS')","(606346, Business Operations Analyst)"
1,0.824317,"(3603, 'Human resources management system HRMS')","(606346, Business Operations Analyst)"
2,0.824258,"(5682, 'OrangeHRM')","(606346, Business Operations Analyst)"


In [90]:
query2 = """
select similarity_score, technology_skill, substr(job_id_title, 2, instr(job_id_title, ',') - 2) as job_id, instr(job_id_title, ',') as comma_pos
from df_skill
"""
df_skill = sqldf(query2, globals())
df_skill[0:3]

,similarity_score,technology_skill,job_id,comma_pos
0,0.833890,"(5625, 'Oracle HRIS')",606346,8
1,0.824317,"(3603, 'Human resources management system HRMS')",606346,8
2,0.824258,"(5682, 'OrangeHRM')",606346,8


In [92]:
query_join = """
select job.[Job ID], job.Agency, skill.technology_skill from df_job_post job left outer join df_skill skill on job.[Job ID] = skill.job_id
"""
sqldf(query_join, globals())

,Job ID,Agency,technology_skill
0,607141,DEPARTMENT OF CITY PLANNING,"(5996, 'PlanGraphics Citywide GIS Utility')"
1,607141,DEPARTMENT OF CITY PLANNING,"(2282, 'Digital Crew Teamwork Project Manager')"
2,607141,DEPARTMENT OF CITY PLANNING,"(5210, 'Municipal geographic management software')"
3,607141,DEPARTMENT OF CITY PLANNING,"(1960, 'Creative Data Solutions DuProcess')"
4,607141,DEPARTMENT OF CITY PLANNING,"(4085, 'JDA Master Planning')"
...,...,...,...
115,580478,BRONX DISTRICT ATTORNEY,"(2161, 'Datair Employee Benefits Systems')"
116,580478,BRONX DISTRICT ATTORNEY,"(2070, 'DPS Software General Litigation')"
117,580478,BRONX DISTRICT ATTORNEY,"(7725, 'Tesseract Human Resources Manager')"
118,580478,BRONX DISTRICT ATTORNEY,"(5651, 'Oracle PeopleSoft Enterprise Human Resources')"
